# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [2]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
import gradio as gr

In [3]:

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:2]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk
Google API Key exists and begins AI


In [6]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()

MODEL_OPTIONS = ["gpt-3.5-turbo", "gpt-4", "ollama"]

In [7]:
system_prompt = "You are an expert Python developer and educator. Provide clear and concise explanations for the given code snippets."

In [24]:
def get_openai_response(model: str, question: str):
    """
    Generates a streaming response from OpenAI's chat completions API.

    Args:
        model: The OpenAI model to use (e.g., "gpt-3.5-turbo", "gpt-4").
        question: The user's query.

    Yields:
        The cumulative response string as it's generated.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ]
    response = ""
    for chunk in openai.chat.completions.create(model=model, messages=messages, stream=True):
        content = chunk.choices[0].delta.content
        if content:
            response += content
            yield response

In [ ]:
def get_ollama_response(question: str, model: str = OLLAMA_MODEL):
    """
    Generates a streaming response from an Ollama model.

    Args:
        question: The user's query.
        model: The Ollama model to use (default: OLLAMA_MODEL).

    Yields:
        The cumulative response string as it's generated.
    """
    import ollama  # Import here to avoid errors if not using Ollama

    response = ""
    for chunk in ollama.chat(model=model, messages=[{"role": "user", "content": question}], stream=True):
        content = chunk['message']['content']
        if content:
            response += content
            yield response

In [31]:
# --- Main Answer Function ---
def answer_question(model: str, question: str):
    """
    Routes the question to the appropriate LLM provider (OpenAI or Ollama)
    and yields the streaming response to Gradio.

    Args:
        model: The selected LLM model.
        question: The user's query.

    Yields:
        The streaming chunks of the response.
    """

    generator = None

    if model in ["gpt-3.5-turbo", "gpt-4", "gpt-4o"]:
        generator = get_openai_response(model, question)
    elif model == "ollama":
        try:
            generator = get_ollama_response(question)
        except ImportError:
            yield "Error: Ollama library not found. Please install it."
            return
        except Exception as e:
            yield f"Error with Ollama: {e}"
            return
    else:
        yield "Selected model is not supported."
        return

    if generator:
        for chunk in generator:
            yield chunk

In [32]:
# --- Gradio Interface ---
with gr.Blocks() as demo:
    gr.Markdown("## Technical Question Answerer")

    with gr.Row():
        model_dropdown = gr.Dropdown(choices=MODEL_OPTIONS, label="Select Model", value=MODEL_OPTIONS[0])
    question_input = gr.Textbox(label="Enter your technical question here:", lines=5)
    answer_output = gr.Textbox(label="AI Explanation", lines=10)
    submit_button = gr.Button("Get Explanation")

    submit_button.click(fn=answer_question, inputs=[model_dropdown, question_input], outputs=[answer_output])


# Launch the interface
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7940

To create a public link, set `share=True` in `launch()`.
